In [1]:
!pip install coiled

  Using cached prompt_toolkit-2.0.10-py3-none-any.whl (340 kB)
  Using cached urllib3-1.24.3-py2.py3-none-any.whl (118 kB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.0.3
    Uninstalling pandas-1.0.3:
      Successfully uninstalled pandas-1.0.3
  Attempting uninstall: distributed
    Found existing installation: distributed 2.22.0
    Uninstalling distributed-2.22.0:
      Successfully uninstalled distributed-2.22.0
  Attempting uninstall: dask
    Found existing installation: dask 2.22.0
    Uninstalling dask-2.22.0:
      Successfully uninstalled dask-2.22.0
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 3.0.4
    Uninstalling prompt-toolkit-3.0.4:
      Successfully uninstalled prompt-toolkit-3.0.4
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.9
    Uninstalling urllib3-1.25.9:
      Successfully uninstalled urllib3-1.25.9


ERROR: apache-airflow 1.10.9 requires flask<2.0,>=1.1.0, which is not installed.
ERROR: apache-airflow 1.10.9 requires unicodecsv>=0.14.1, which is not installed.
ERROR: apache-airflow 1.10.9 requires werkzeug<1.0.0, which is not installed.
ERROR: sparkmagic 0.12.9 requires nose, which is not installed.
ERROR: mlflow 0.9.1 requires Flask, which is not installed.
ERROR: hdijupyterutils 0.12.9 requires jupyter>=1, which is not installed.
ERROR: hdijupyterutils 0.12.9 requires nose, which is not installed.
ERROR: datashader 0.8.0 requires datashape>=0.5.1, which is not installed.
ERROR: datashader 0.8.0 requires scikit-image, which is not installed.
ERROR: modin 0.7.0 has requirement pandas==0.25.3, but you'll have pandas 1.1.2 which is incompatible.
ERROR: h2o-pysparkling-2-4 2.4.10 has requirement pyspark<=2.4.0,>=2.4.0, but you'll have pyspark 2.3.1 which is incompatible.
ERROR: apache-airflow 1.10.9 has requirement jinja2<2.11.0,>=2.10.1, but you'll have jinja2 2.11.1 which is incompa

In [3]:
!coiled login --token 1166cf82461ae2fbf7f41884ef1a54de1a3cd212

Credentials have been saved at C:\Users\argenisleon\.config\dask\coiled.yaml


In [1]:
import coiled

In [ ]:
coiled.create_software_environment(name='zarr', conda=['xarray', 'zarr','dask=2.2.23','distributed'])

In [6]:
coiled.create_cluster_configuration(
    name="bumblebee",
    worker_memory="16 GiB",
    worker_cpu=4,
    scheduler_memory="4 GiB",
    scheduler_cpu=1,
    software="coiled/default"
)

In [10]:
cluster.name #salvar en bumblebee

'argenisleon-82e8c2c8-7'

In [9]:
coiled.list_clusters()

{'argenisleon-82e8c2c8-7': {'id': 798,
  'status': 'running',
  'account': 'argenisleon',
  'dashboard_address': 'http://ec2-18-221-23-232.us-east-2.compute.amazonaws.com:8787',
  'configuration': 126,
  'address': 'tls://ec2-18-221-23-232.us-east-2.compute.amazonaws.com:8786'}}

In [11]:
cluster = coiled.Cluster(configuration='bumblebee')

Creating Cluster. This takes about a minute ...Checking environment images
Valid environment image found


In [ ]:
cluster.name # argenisleon-82e8c2c8-7

In [ ]:
coiled.list_clusters()  # {"argenisleon-82e8c2c8-7":{...}}

In [5]:
from dask.distributed import Client
client = Client(name=cluster.name)
print('Dashboard:', client.dashboard_link)

Dashboard: http://ec2-18-219-253-148.us-east-2.compute.amazonaws.com:8787/status


C:\Users\argenisleon\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\argenisleon\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\argenisleon\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)
C:\Users\argenisleon\Anaconda3\lib\site-packages\distributed\client.py:1137: VersionMismatchWarning: Mismatched versions found

+-------------+---------------+---------------+---------------+
| Package     | client        | scheduler     | workers       |
+-------------+---------------+---------------+---------------+
| blosc       | None          | 1.9.2         | 1.9.2         |
| cloudpickle | 1.5.0         | 1.6.0         | 1.6.0         |
| dask        | 2.25.0        | 2.23.0        | 2.23.0        |
| distributed | 2.25.0        | 2.26.0        | 2.26.0        |
| lz4         | None  

In [4]:
dir(coiled)

['Cloud',
 'Cluster',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__getattr__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_version',
 'cli',
 'cluster',
 'compatibility',
 'core',
 'create_cluster',
 'create_cluster_configuration',
 'create_job_configuration',
 'create_software_environment',
 'delete_cluster',
 'delete_cluster_configuration',
 'delete_job_configuration',
 'delete_software_environment',
 'inspect',
 'install',
 'list_cluster_configurations',
 'list_clusters',
 'list_job_configurations',
 'list_jobs',
 'list_software_environments',
 'start_job',
 'stop_job',
 'upload',
 'utils']

In [19]:
import dask.dataframe as dd

df = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-*.csv",
    dtype={
        "payment_type": "UInt8",
        "VendorID": "UInt8",
        "passenger_count": "UInt8",
        "RatecodeID": "UInt8",
    },
    storage_options={"anon": True},
    blocksize="16 MiB",
).persist()

df.groupby("passenger_count").tip_amount.mean().compute()

KeyboardInterrupt: 